In [1]:
# public packages
import andes
import ams
import cvxpy as cp
import numpy as np
import gurobipy

# local packages
import anci_fun as af

# v1 try to include dynamic constraints using reduced input/output NN model

# ------ nn information ------
# input:   [M, D]
# nn_diam: [16, 128, 1]

# Para preparetion

In [2]:
# neural network parameters 
name_rocof  = 'NN_rocof.pth'
name_fnadir = 'NN_fnadir.pth'
name_angle  = 'NN_angle.pth'
name_eig    = 'NN_eig.pth'

nn_weights_rocof, nn_bias_rocof, nn_diam_rocof    = af.extract_nn_para(name_rocof) # three list
nn_weights_fnadir, nn_bias_fnadir, nn_diam_fnadir = af.extract_nn_para(name_fnadir) 
nn_weights_angle, nn_bias_angle, nn_diam_angle    = af.extract_nn_para(name_angle)
nn_weights_eig, nn_bias_eig, nn_diam_eig          = af.extract_nn_para(name_eig) 

In [3]:
# MD parameters
vsg_num = 8

# lb_M = np.array([0, 0, 0, 0, 0, 0, 0, 0])
lb_M = np.array([0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05])
ub_M = np.array([6, 6, 6, 6, 6, 6, 6, 6])

# lb_D = np.array([0, 0, 0, 0, 0, 0, 0, 0])
lb_D = np.array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01])*10
ub_D = np.array([4, 4, 4, 4, 4, 4, 4, 4])

In [4]:
# boundary rocof
lb_rocof = np.array([-0.5])
ub_rocof = np.array([ 0.4])

# boundary fnadir
lb_fnadir = np.array([-0.6])
ub_fnadir = np.array([ 0.6])

# boundary angle
lb_angle = np.array([-3.14])
ub_angle = np.array([ 3.14])

# boundary eig
lb_eig = np.array([-100])
ub_eig = np.array([ 0.001])

# Load ams model

In [5]:
ssp = ams.load('ams_case123_REGCV1_v5.xlsx')

ssp.LDOPF.run()

ssp.LDOPF.om.mdl.solve()

ssp.LDOPF.om.vars

LDOPF solved as optimal in 0.0798 seconds with exit code 0.


OrderedDict([('pg', Variable((11,))),
             ('pn', Variable((117,))),
             ('qg', Variable((11,))),
             ('qn', Variable((117,))),
             ('vsq', Variable((117,))),
             ('pl', Variable((116,))),
             ('ql', Variable((116,)))])

# Define new variables, constraints, and obj

In [6]:
# M and D

M = cp.Variable(vsg_num, name="M")
D = cp.Variable(vsg_num, name="D")

constraints_M = [M >= lb_M, M <= ub_M]
constraints_D = [D >= lb_D, D <= ub_D]

In [7]:
# data for the neural network linearization
pg    = ssp.LDOPF.om.pg

x1    = cp.hstack([M, D])  # this version doesn't incelu load scaling factor

w1_rocof    = nn_weights_rocof[0]
w2_rocof    = nn_weights_rocof[1]

w1_fnadir   = nn_weights_fnadir[0]
w2_fnadir   = nn_weights_fnadir[1]

w1_angle    = nn_weights_angle[0]
w2_angle    = nn_weights_angle[1]

w1_eig      = nn_weights_eig[0]
w2_eig      = nn_weights_eig[1]

b1_rocof    = nn_bias_rocof[0]
b2_rocof    = nn_bias_rocof[1]

b1_fnadir   = nn_bias_fnadir[0]
b2_fnadir   = nn_bias_fnadir[1]

b1_angle    = nn_bias_angle[0]
b2_angle    = nn_bias_angle[1]

b1_eig      = nn_bias_eig[0]
b2_eig      = nn_bias_eig[1]

In [8]:
# neural network linization
z_rocof  = w1_rocof @ x1 + b1_rocof
z_fnadir = w1_fnadir @ x1 + b1_fnadir
z_angle  = w1_angle @ x1 + b1_angle
z_eig    = w1_eig @ x1 + b1_eig

y_rocof  = w2_rocof @ z_rocof + b2_rocof
y_fnadir = w2_fnadir @ z_fnadir + b2_fnadir
y_angle  = w2_angle @ z_angle + b2_angle
y_eig    = w2_eig @ z_eig + b2_eig

constraints_nn = [
                    y_rocof >= lb_rocof,
                    y_rocof <= ub_rocof,
                    y_fnadir >= lb_fnadir,
                    y_fnadir <= ub_fnadir,
                    y_angle >= lb_angle,
                    y_angle <= ub_angle,
                    y_eig <= ub_eig
                ]

In [9]:
# extract original ams constraints
constriants_ams = [ssp.LDOPF.om.constrs[cname] for cname in ssp.LDOPF.om.constrs.keys()]

In [10]:
constraints = constraints_M + constraints_D + constraints_nn + constriants_ams

# constraints = constriants_ams + constraints_M + constraints_D

# Formulate new problem

pg = ['PV_1',
 'PV_10',
 'PV_11',
 'PV_2',
 'PV_3',
 'PV_4',
 'PV_5',
 'PV_6',
 'PV_7',
 'PV_8',
 'PV_9']

 SG: PV_1, PV_2, PV_3 (slack bus)

 IBR: PV_4, PV_5, ..., PV_11

In [11]:
c2 = np.array([0.01, 0, 0, 0.01, 0.02, 0, 0, 0, 0, 0, 0])*100
c1 = np.array([  20, 0, 0,   20,   30, 0, 0, 0, 0, 0, 0])*10

c_M  = np.array([0.02, 0.012, 0.5, 0.014, 0.06, 0.07, 0.015, 0.02])

c_D  = np.array([0.01, 0.01, 0.03, 0.01, 0.01, 0.01, 0.02, 0.01])*3

pg_power = pg**2

cost = cp.sum(c2 @ pg_power + c1 @ pg + c_M @ M + c_D @ D)

obj = cp.Minimize(cost)

In [12]:
# Create the optimization problem
prob = cp.Problem(obj, constraints)

# Solve the optimization problem

# prob.solve(solver=cp.GUROBI, verbose=True)

prob.solve(solver=cp.ECOS_BB, verbose=True)

# prob.solve()

                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Sep 11 10:42:45 PM: Your problem has 621 variables, 24 constraints, and 0 parameters.
(CVXPY) Sep 11 10:42:45 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Sep 11 10:42:45 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Sep 11 10:42:45 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Sep 11 10:42:45 PM: Compiling problem (target solver=ECOS_BB).
(CVXPY) Sep 11 10:42:45 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffi

20.40157932472802

In [13]:
ssp.LDOPF.om.pg.value

array([0.02    , 0.034125, 0.034125, 0.02    , 0.04    , 0.034125,
       0.034125, 0.034125, 0.034125, 0.034125, 0.034125])

In [14]:
M.value

array([0.05      , 0.05      , 0.05      , 0.05      , 1.81181949,
       0.05      , 0.05      , 0.05      ])

In [15]:
D.value

array([0.1       , 0.1       , 0.1       , 3.64400516, 0.1       ,
       0.1       , 0.1       , 4.        ])

# 